In [2]:
freq1, freq2, offsets1, offsets2 = np.random.rand(4, 1000, 1)

In [8]:
np.newaxis

In [9]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time- offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 +20))
    series += 0.2 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[np.newaxis].astype(np.float32)

In [ ]:
n_steps = 50
series = gen